<a href="https://colab.research.google.com/github/xilo94/Big_Data_ETL/blob/main/part_two_electronics_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-23 18:19:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-12-23 18:19:34 (10.7 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://bigetlxm.s3.amazonaws.com/amazon_reviews_us_Mobile_Electronics_v1_00.tsv"
spark.sparkContext.addFile(url)

# df = spark.read.format("tsv").option("header", "true").load("csvfile.csv")

userdf = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv"), sep="\t", header=True, inferSchema=True)


In [5]:
# Get the number of rows in the DataFrame.
userdf.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.


review_id_df = userdf.select("marketplace","customer_id", "review_id", "review_headline", "review_date")

review_id_df.show()


+-----------+-----------+--------------+--------------------+-----------+
|marketplace|customer_id|     review_id|     review_headline|review_date|
+-----------+-----------+--------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|         Very Happy!| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|           five star| 2015-08-31|
|         US|   51469641|R2Y0MM9YE6OP3P|        great cables| 2015-08-31|
|         US|    4332923| RRB9C05HDOD4O|Work very well bu...| 2015-08-31|
|         US|   44855305|R26I2RI1GFV8QG|Cameras has batte...| 2015-08-31|
|         US|    7846966| RY8DDL22YG4R5|It appears to be ...| 2015-08-31|
|         US|   21299354|R2AT2426ZHFUHH|Didn't love the f...| 2015-08-31|
|         US|   28902968|R3RRXU2R23NMQ9|          Five Stars| 2015-08-31|
|         US|    2074438|R250PR8VJUZ62F|          Four Stars| 2015-08-31|
|         US|   48701722| RBEMQ29WJBHYG|  I LOVE my recorder| 2015-08-31|
|         US|    2990247|R19VVIUT4BZCM

In [7]:
review_id_df = review_id_df.dropna() 

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df = userdf.select("product_id", "product_title")
products_df = products_df.dropna()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00MC4CED8|BlackVue DR600GW-PMP|
|B00OQMFG1Q|GENSSI GSM / GPS ...|
|B00QERR5CY|iXCC Multi pack L...|
|B00QUFTPV4|abcGoodefg® FBI C...|
|B0067XVNTG|Generic Car Dashb...|
|B00KA6CCVY|Aweek® Air Acoust...|
|B00MJCDPM2|Sentey LS-4460 B-...|
|B00ET5AWBY|iPad Car Headrest...|
|B00Z9K379C|ANWING Waterproof...|
|B00YO3UYXW|Jensen MCR-100 Ca...|
|B011W26BU2|Bluetooth Speaker...|
|B00H8PI78C|15Pcs DC 5V 85dB ...|
|B00M49C6MK|      Sentey Ls 4162|
|B011Q291PK|IcePACK 8 Can Ins...|
|B00W6OLUGQ|DTK® AMI MDI MMI ...|
|B008MY1F8W|INPA K+CAN K+DCAN...|
|B00KW3XNUE|Magicfly Wireless...|
|B009NGR0AK|Humminbird XNT 9 ...|
|B011W26BU2|Bluetooth Speaker...|
|B00PAP68WO|Foxnovo 3.7V 380m...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

customers_df = userdf.select("customer_id", "review_id", "review_headline",)

customers_df = customers_df.groupBy('customer_id').agg({"review_id": "count"})
customers_df.show()


+-----------+----------------+
|customer_id|count(review_id)|
+-----------+----------------+
|     176073|               1|
|   20992492|               1|
|    1397686|               1|
|   50371142|               1|
|     235096|               1|
|   12320806|               1|
|   19707203|               1|
|   15207172|               1|
|     538703|               1|
|   11223135|               1|
|   31447151|               1|
|   35961247|               1|
|   17835519|               1|
|   15404885|               1|
|   10182610|               1|
|   52399092|               1|
|   17133428|               1|
|   27870423|               1|
|   11574168|               1|
|   30685607|               1|
+-----------+----------------+
only showing top 20 rows



## Create the "vine_table".

In [10]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = userdf.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8MEA6IGAHO0B|          5|            0|          0|   N|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|
| RRB9C05HDOD4O|          4|            0|          0|   N|
|R26I2RI1GFV8QG|          2|            0|          0|   N|
| RY8DDL22YG4R5|          3|            0|          1|   N|
|R2AT2426ZHFUHH|          3|            0|          1|   N|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|
|R250PR8VJUZ62F|          4|            0|          2|   N|
| RBEMQ29WJBHYG|          5|          164|        168|   N|
|R19VVIUT4BZCMT|          5|            2|          2|   N|
|R1DT8JJUQHUKUL|          3|            0|          1|   N|
| RLIL6S9OGM3YA|          5|            0|          0|   N|
|R34EZZ68VYPHO0|          1|            

# Load

In [11]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cyfynzqdeakg.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", "password": "postgres", "driver":"org.postgresql.Driver"}

In [12]:
# Write review_id_df to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)


In [13]:
# Write products_df to table in RDS


review_id_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)

In [14]:
# Write customers_df to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='customers_df', mode=mode, properties=config)


In [15]:
# Write vine_df to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='vine_df', mode=mode, properties=config)
